In [4]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd
from transformers import pipeline
from datetime import datetime, timedelta
import re 
import torch

c:\Users\saxen\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
torch.__version__

'2.6.0+cu126'

In [6]:
tickertape_url = "https://www.tickertape.in/stocks/"
tickers = ['PNBK','MRF','RAIV','NTPC']
news_list = []
pipe = pipeline("text-classification", model="ProsusAI/finbert")

Device set to use cuda:0


In [7]:
def convert_relative_date(relative_date):
    today = datetime.today()

    match = re.match(
        r'(\d+) (minute|hour|day|week|month|year)s? ago', relative_date.lower())

    if match:
        num = int(match.group(1))
        unit = match.group(2)

        if unit == "minute":
            return today - timedelta(minutes=num)
        elif unit == "hour":
            return today - timedelta(hours=num)
        elif unit == "day":
            return today - timedelta(days=num)
        elif unit == "week":
            return today - timedelta(weeks=num)
        elif unit == "month":
            return today - timedelta(days=num * 30)
        elif unit == "year":
            # Approximate 1 year = 365 days
            return today - timedelta(days=num * 365)

    return today

In [8]:
for ticker in tickers:
    url = tickertape_url + ticker
    req = Request(url=url, headers={'user-agent': 'my-app'})
    response = urlopen(req)
    html = BeautifulSoup(response, 'html.parser')

    news_tab = html.find(id='news-tab')
    if news_tab:
        headlines = news_tab.find_all(
            'span', class_='jsx-1669302504 text-primary headline mb8 heading')
        dates = news_tab.find_all(
            'span', class_='jsx-1669302504 text-grey caption-style all-uppercase')

        for headline, date in zip(headlines, dates):
            news_list.append([
                ticker,
                headline.text.strip(),
                convert_relative_date(date.text.strip()).strftime(
                    '%Y-%m-%d %H:%M:%S')
            ])

In [9]:
df = pd.DataFrame(news_list, columns=['Ticker', 'Title', 'Date'])

In [10]:
df.head()

,Ticker,Title,Date
0,PNBK,Punjab National Bank consolidated net profit r...,2025-02-07 14:29:29
1,PNBK,Punjab National Bank rallies as Q3 PAT doubles...,2025-02-07 14:29:29
2,PNBK,Punjab National Bank to convene board meeting,2025-01-31 14:29:29
3,PNBK,PNB rises as domestic deposits climb 14% YoY i...,2025-01-15 14:29:29
4,PNBK,Punjab National Bank announces cessation of MD...,2025-01-15 14:29:29


In [11]:
df.tail()

,Ticker,Title,Date
35,NTPC,NTPC to hold board meeting,2025-01-24 14:29:32
36,NTPC,"NTPC Ltd soars 2.96%, rises for fifth straight...",2025-01-24 14:29:32
37,NTPC,NTPC commissions further 25 MW of Gujarat Sola...,2025-01-24 14:29:32
38,NTPC,"NTPC Ltd soars 1.48%, Gains for third straight...",2025-01-17 14:29:32
39,NTPC,NTPC commissions two solar projects aggregatin...,2025-01-15 14:29:32


In [32]:
text = "The Punjab National bank recorded loss of 2400Cr in the last fiscal year"
pipe(text)[0]['label']

'negative'

In [31]:
pipe(text)[0]['score']

0.9625182151794434

In [37]:
def f(Title): return pd.Series([pipe(Title)[
    0]['label'], pipe(Title)[0]['score']])


df[['label', 'confidence']] = df['Title'].apply(f)

In [39]:
df

,Ticker,Title,Date,label,confidence
0,PNBK,Punjab National Bank consolidated net profit r...,2025-02-07 14:29:29,positive,0.932558
1,PNBK,Punjab National Bank rallies as Q3 PAT doubles...,2025-02-07 14:29:29,positive,0.808500
2,PNBK,Punjab National Bank to convene board meeting,2025-01-31 14:29:29,neutral,0.936114
3,PNBK,PNB rises as domestic deposits climb 14% YoY i...,2025-01-15 14:29:29,positive,0.931605
4,PNBK,Punjab National Bank announces cessation of MD...,2025-01-15 14:29:29,neutral,0.711983
5,PNBK,Punjab National Bank receives affirmation in c...,2024-12-16 14:29:29,neutral,0.533670
6,PNBK,Punjab National Bank up for third straight ses...,2024-12-16 14:29:29,positive,0.753224
7,PNBK,Punjab National Bank consolidated net profit r...,2024-11-16 14:29:29,positive,0.929492
8,PNBK,Punjab National Bank announces board meeting date,2024-11-16 14:29:29,neutral,0.844728
9,PNBK,Punjab National Bank launches QIP; floor price...,2024-10-17 14:29:29,neutral,0.894893
